In [1]:
%pwd

'C:\\python'

In [2]:
%cd C:\Users\user\Desktop\구구즈 - 문화누리카드

C:\Users\user\Desktop\구구즈 - 문화누리카드


In [ ]:
'''
%cd C:/python

! pip3 install numpyencoder 

import numpy as np
import pandas as pd

import random
import json
import os
import sys
import urllib.request

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

! pip3 install numpyencoder 

'''

#font

plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['axes.grid'] = False

prop = fm.FontProperties(fname='C://python//fonts')
font_dir = ['./fonts']

plt.rcParams['font.family'] = prop.get_family()
for font in fm.findSystemFonts(fontpaths=font_dir):
    print(fm.FontProperties(fname=font).get_name())
    fm.fontManager.addfont(font)

mpl.rcParams['font.family'] = 'D2Coding'
print(mpl.rcParams['font.family'])



# 문화누리카드 오프라인 가맹점 Data csv 불러오기
offline = pd.read_csv("99z/datas/culture_nuri_card_offline_shop.csv", encoding = "cp949")  # raw data

# 결측값 확인 -> 없음
offline.isna().sum()  

# 지역별 가맹점 수
ser_re_off = offline.지역.value_counts()  

# 총 데이터 수
offline.지역.value_counts().sum()

offline.지역.unique()
# 지역 이름에 공백 존재함을 확인

# 공백 제거 (최초 1번만 실행해야 함!) #
offline.지역 = offline.지역.str[:2]
offline.지역.unique()


### 전국 기준 data
result= []
for cat in offline["지역"].unique():
    data = offline[offline["지역"] == cat]["분류"].value_counts(True).to_frame() *100
    data["지역"] = cat
    result.append(data)

# 서울 / 경기만 필터링한 Data frame
offline_reg = offline[offline['지역'].isin(["서울", "경기"])]
offline_reg

# 서울/경기 지역 분류별 가맹점 수 (Series)
offline_reg.분류.value_counts()

'''--------------------data preprocessing-----------------------'''




'''----------------<class>--------------------'''
#NAVER 데이터랩(검색어트렌드) - 통합검색어 트렌드 조회

class naverDataLabOpenAPI():
    '''
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    '''
    
    def __init__(self, client_id, client_secret):
        '''
        인증키 설정 및 검색어 그룹 초기화
        '''
        self.client_id = client_id
        self.client_secret = client_secret
        self.keywordGroups = []
        self.url = url
        
    def add_keyword_groups(self, group_dict):
        '''
        검색어  그룹 추가
        self.keywordGroups 라는 빈 리스트가 생긴 상태
        검색어 그룹을 미리 Dictionary 객체에 정의한 후
        아래 메서드(def)를 통해 self.keywordGroups 리스트에 추가한다.
        '''
        keyword_group = {
            'groupName' : group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_group)
        print(f'>>> Num of KeywordGroups: {len(self.keywordGroups)}')
        
    def get_data(self, startDate, endDate, timeUnit, device, ages, gender):
        '''
        데이터 요청 기능
        요청 파라미터 설정 후 아래 메서드에 입력하면
        DataFrame 형태로 데이터 불러올 수 있다.
        '''        
        
        #Request body
        body = json.dumps({
            'startDate': startDate,
            'endDate': endDate,
            'timeUnit': timeUnit,
            'keywordGroups': self.keywordGroups,
            'device': device,
            'ages': ages,
            'gender': gender
        }, ensure_ascii=False, cls= NumpyEncoder)
        
        #Results
        request = urllib.request.Request(self.url)
        request.add_header('X-Naver-Client-Id', client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)    
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            #Json result
            response_text = response.read()
            try:
                result = json.loads(response_text)
            except:
                print(response_text)
            
            df = pd.DataFrame(result['results'][0]['data'])[['period']]
            for i in range(len(self.keywordGroups)):
                tmp = pd.DataFrame(result['results'][i]['data'])
                tmp = tmp.rename(columns = {'ratio': result['results'][i]['title']})
                df = pd.merge(df, tmp, how='left', on=['period'])
            self.df = df.rename(columns={'period': '날짜'})
            self.df['날짜'] = pd.to_datetime(self.df['날짜'])
        else:
            print("Error Code:" + rescode)
        return self.df
    
    def plot_monthly_trend(self):
        '''
        월 별 검색어 트렌드 그래프 출력
        '''
        df = self.df.copy()
        df_0 = df.groupby(by=[df['날짜'].dt.year, df['날짜'].dt.month]).mean().droplevel(0).reset_index().rename(columns={'날짜':'월'})
        df_1 = df.groupby(by=[df['날짜'].dt.year, df['날짜'].dt.month]).mean().droplevel(1).reset_index().rename(columns={'날짜':'년도'})
        
        df = pd.merge(df_1[['년도']], df_0, how='left', left_index=True, right_index=True)
        df['날짜'] = pd.to_datetime(df[['년도','월']].assign(일=1).rename(columns={'년도':'year','월':'month','일':'day'}))
        
        colList = df.columns.drop(['날짜', '년도', '월'])
        n_col = len(colList)
        
        fig = plt.figure(figsize=(12,6))
        plt.title('월 별 검색어 트렌드', size=20, weight='bold')
        for i in range(n_col):
            sns.lineplot(x=df['날짜'], y=df[colList[i]], label=colList[i])
        plt.legend(loc='upper right')
        plt.ylabel('%')
        
        return fig


'''----------------------------</class>---------------------------'''


#API 인증 정보 설정
client_id = "QLTrFRPqtguWpbahbauE"
client_secret = "jRVI1MzywQ"
url = "https://openapi.naver.com/v1/datalab/search"


#요청 파라미터 설정
'''
startDate : str :조회 기간 시작 날짜 (yyyy-mm-dd 형식, 2016-01-01부터 가능)
endDate : str : 조회 기간 종료 날짜 (yyyy-mm-dd 형식)
timeUnit : str :구간 단위, date(일간)/week(주간)/month(월간)
device : str : 검색 환경 조건, 설정안함(모든환경)/pc(PC검색추이),mo(모바일검색추이)
ages : list : 검색 사용자의 연령, 설정안함(모든 연령), [1:11] 이런 식인 듯
    1: 0∼12세 / 2: 13∼18세 /3: 19∼24세 /4: 25∼29세/5: 30∼34세/6: 35∼39세
    7: 40∼44세/8: 45∼49세/9: 50∼54세/10: 55∼59세/11: 60세 이상
gender : 검색 사용자의 성별, 설정안함(모든 성별), m:남성, f:여성
'''
startDate = '2016-01-01'
endDate = '2023-03-17'
timeUnit = 'date'
device = ''
ages = []
gender = ''




'''---------------------data preprocessing 2-------------------'''

#편한 인덱싱을 위해 '분류'를 인덱스로 설정
offline2 = offline.set_index('분류')
offline2.head(3)

#'분류'의 종류 list
oi = list(offline2.index.unique())

#'분류' 별 가맹점 20개 랜덤 추출
random.sample(list(offline2.loc[offline2.index.unique()[0]]['가맹점명']),20)

'''-------------------------calling class--------------------------'''


random.seed(0)
keyword_group_set = {
    'keyword_group_1': {'groupName': oi[0], 'keywords': random.sample(list(offline2.loc[offline2.index.unique()[0]]['가맹점명']),20)},
    'keyword_group_2': {'groupName': oi[1], 'keywords': random.sample(list(offline2.loc[offline2.index.unique()[1]]['가맹점명']),20)},
    'keyword_group_3': {'groupName': oi[2], 'keywords': random.sample(list(offline2.loc[offline2.index.unique()[2]]['가맹점명']),20)},
    'keyword_group_4': {'groupName': oi[3], 'keywords': random.sample(list(offline2.loc[offline2.index.unique()[3]]['가맹점명']),20)},
    'keyword_group_5': {'groupName': oi[4], 'keywords': random.sample(list(offline2.loc[offline2.index.unique()[4]]['가맹점명']),20)},
}



naver = naverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

for key in keyword_group_set:
    naver.add_keyword_groups(keyword_group_set[key])
    
df1 = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)

naver.plot_monthly_trend()
plt.show()